### Download/save data

In [1]:
import wget  # pip install wget if needed
import scipy.io
import scipy.sparse

meta_http = "https://kleintools.hms.harvard.edu/paper_websites/state_fate2020/stateFate_inVitro_metadata.txt.gz"
meta_out = "metadata.txt.gz"

clone_mat_http = "https://kleintools.hms.harvard.edu/paper_websites/state_fate2020/stateFate_inVitro_clone_matrix.mtx.gz"
clone_mat_out = "clone_matrix.mtx.gz"

wget.download(meta_http, meta_out, bar=False)
wget.download(clone_mat_http, clone_mat_out, bar=False)
mat = scipy.io.mmread(clone_mat_out)

# save as .npz for quicker reading
scipy.sparse.save_npz("clone_matrix.npz", mat)

### Read metadata 

In [2]:
import numpy as np
import pandas as pd

mat = scipy.sparse.load_npz("clone_matrix.npz").toarray()
meta_df = pd.read_csv("metadata.txt.gz", sep="\t")

### Annotate lineage-traced (LT) cells in `meta_df`

In [3]:
tmp = np.zeros(len(mat))
LT_idx = np.where(mat.sum(1) > 0)[0]
tmp[LT_idx] = 1
meta_df["LT"] = tmp

In [4]:
X_clone = mat[mat.sum(1) > 0]
X_clone.shape

(49302, 5864)

### Annotate clonally barcoded cells with their corresponding lineage (one lineage per cell, max)

In [5]:
cell_idx, clone_idx = np.where(X_clone > 0)
tmp = np.full(len(meta_df), np.nan)
tmp[cell_idx] = clone_idx
meta_df["clone_idx"] = tmp

### Day 2 cells that have LT barcodes

### Grab clone index: `clone_idx`

* Each day 2 cell that has a clonal barcode will belong to a clonal lineage (numbered according to position in `X_clone` and annotated in `meta_df`. 
* Regardless of how many d2 cells are observed to have a given lineage, we will count the number of cells observed in each of hte 11 possible annotations (including `undiff`) at d4 and d6. These will be summed, producing a ` clone x fate ` outcome matrix. We can then row-normalize such that each row sums to 1, and the relative percentage of fate produced (counted at d4, d6) will then be represented.

In [6]:
def _count_downstream(df, key):
    return (
        df.loc[df["Time point"].isin([4, 6])]
        .groupby("Cell type annotation")
        .count()[key]
    )


def _d2_clone_progenitor_counts(meta_df):

    """
    group by clone idx, filter clones containing only a
    single timepoint or those without a cell at d2. Then,
    count the cells at d4, d6 corresponding to that clone idx.
    if the above conditions are not satisfied, return an empty series
    """

    key = meta_df.columns[1]

    d2_clone_progenitor_dict = {}

    for clone, clone_df in meta_df.groupby("clone_idx"):
        clone_time = clone_df["Time point"].unique()
        if (2 in clone_time) and (len(clone_time) > 1):
            d2_clone_progenitor_dict[clone] = _count_downstream(clone_df, key)
        else:
            d2_clone_progenitor_dict[clone] = pd.Series(
                np.array([]), name="Cell barcode"
            )

    return pd.DataFrame.from_dict(d2_clone_progenitor_dict).fillna(0).T

In [7]:
counted_clones = _d2_clone_progenitor_counts(meta_df)
counted_clones

,Baso,Ccr7_DC,Eos,Erythroid,Lymphoid,Mast,Meg,Monocyte,Neutrophil,Undifferentiated,pDC
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,5.0,0.0
2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,5.0,7.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5859.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5861.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5862.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
counted_clones.shape

(5864, 11)

Now for each d2 cell, you can map the values in `counted_clones` by the clonal lineage they belong to:

In [9]:
counted_clones_ = counted_clones.reset_index().rename({"index": "clone_idx"}, axis=1)
meta_df = meta_df.merge(counted_clones_, on="clone_idx")
d2_clonal_cells = meta_df.loc[meta_df["clone_idx"].notna()].loc[
    meta_df["Time point"] == 2
]
print(d2_clonal_cells.shape)
d2_clonal_cells.head()

(14778, 21)


,Library,Cell barcode,Time point,Starting population,Cell type annotation,Well,SPRING-x,SPRING-y,LT,clone_idx,...,Ccr7_DC,Eos,Erythroid,Lymphoid,Mast,Meg,Monocyte,Neutrophil,Undifferentiated,pDC
11194,d2_3,AAATTCCG-GAGGCTGA,2.0,Lin-Kit+Sca1-,Undifferentiated,0,-1125.399,504.973,0.0,2131.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,5.0,0.0
11195,d2_3,CTGGGTAT-AAGTAATC,2.0,Lin-Kit+Sca1-,Undifferentiated,0,364.161,-603.608,0.0,2131.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,5.0,0.0
11196,d2_3,TAGATCAA-ATCTTTGT,2.0,Lin-Kit+Sca1-,Undifferentiated,0,993.005,-195.602,0.0,2131.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,5.0,0.0
11197,d2_1,CGACATTT-AGAACGGG,2.0,Lin-Kit+Sca1-,Ccr7_DC,0,1732.006,1374.363,0.0,2131.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,5.0,0.0
11198,d2_1,ACGCTTAA-AATGTTTG,2.0,Lin-Kit+Sca1-,Undifferentiated,0,503.878,-470.566,0.0,2131.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,5.0,0.0


Get the 11 unique cell fates and save them in a variable called `cell_fates`:

In [10]:
cell_fates = meta_df["Cell type annotation"].unique()
cell_fates.shape

(11,)

Calculate the fate bias and normalized fate bias for each d2 cell with non-zero fate sums:

In [11]:
d2_fate_biases = d2_clonal_cells[cell_fates]
d2_fate_biases = d2_fate_biases[d2_fate_biases.sum(1) > 0]
d2_fate_biases

,Undifferentiated,Neutrophil,Monocyte,Baso,Erythroid,Mast,Meg,Eos,Ccr7_DC,Lymphoid,pDC
11194,5.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11195,5.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11196,5.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11197,5.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11198,5.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
48601,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
48602,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
48605,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48606,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll save the raw array as `X_fate`:

In [12]:
X_fate = d2_fate_biases.values / d2_fate_biases.sum(1).values[:, np.newaxis]

In [13]:
cell_idx = d2_fate_biases.index
cell_idx.shape, X_fate.shape

((12229,), (12229, 11))

### Save the results

In [14]:
d2_fate_biases.to_csv("LARRY.in_vitro.d2_fate_biases.csv")
np.save("LARRY.in_vitro.X_fate", X_fate)